In [1]:


import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm


# Parameters 
S0 = 25520.60  # Initial asset price
K = 25100.0    # Strike price (from NSE data)
T = 86.0 / 365  # Time to maturity in years (~3 months)
r = 0.068      # Risk-free rate from 91-day T-Bill
sigma = 0.0976 # Implied volatility (9.76%)
N = 120        # Number of time steps
n_sweeps = 100000  # Reduced for faster debugging
n_paths = 10000   # Number of paths for Heston
N_steps = 100     # Time steps for Heston


In [2]:
# Path Integral Monte Carlo (GPU)
def black_scholes_action_gpu(paths, dt, sigma, r, p=2.0, gamma=1.0):
    X = cp.log(paths)
    dX_dt = cp.diff(X, axis=1) / dt
    mu = r - 0.5 * sigma**2
    L = ((dX_dt - mu)**p) / (2 * sigma**p)
    S = cp.sum(L**gamma, axis=1) * dt
    return S

def payoff_gpu(S, K, kind="call"):
    return cp.maximum(S - K, 0) if kind == "call" else cp.maximum(K - S, 0)

def metropolis_hastings_gpu(S0, K, T, r, sigma, N, n_sweeps, tau_corr, kind="call", batch_size=1000):
    dt = T / (N - 1)
    X0 = np.log(S0)
    mu = r - 0.5 * sigma**2
    X_T = X0 + mu * T
    initial_path = cp.exp(cp.linspace(X0, X_T, N))
    paths = cp.tile(initial_path, (batch_size, 1))
    current_action = black_scholes_action_gpu(paths, dt, sigma, r)

    payoffs = []
    perturb_scale = 0.05
    for sweep in range(n_sweeps):
        indices = cp.random.choice(cp.arange(1, N), size=N//2, replace=False)
        perturb = cp.random.normal(0, perturb_scale, size=(batch_size, len(indices)))
        new_paths = paths.copy()
        new_paths[:, indices] += perturb
        new_paths = cp.maximum(new_paths, 1e-6)
        new_action = black_scholes_action_gpu(new_paths, dt, sigma, r)
        dS = new_action - current_action
        accept = (dS <= 0) | (cp.random.rand(batch_size) < cp.exp(-dS))
        paths = cp.where(accept[:, None], new_paths, paths)
        current_action = cp.where(accept, new_action, current_action)
        if sweep % tau_corr == 0:
            payoff_vals = payoff_gpu(paths[:, -1], K, kind)
            payoffs.append(cp.asnumpy(payoff_vals))

    payoffs = np.concatenate(payoffs)
    discounted = np.exp(-r * T) * np.mean(payoffs)
    stderr = np.std(payoffs) / np.sqrt(len(payoffs))
    return discounted, stderr
    # Add this function to validate results
def black_scholes_analytic(S0, K, T, r, sigma, call_put="call"):
    d1 = (np.log(S0/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    if call_put == "call":
        price = S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        price = K*np.exp(-r*T)*norm.cdf(-d2) - S0*norm.cdf(-d1)
    return price


In [3]:
# Heston Simulation (Mixing MC)
def simulate_heston_paths(S0, v0, params, T, N_steps, n_paths, K, kind="call"):
    dt = T / N_steps
    S_paths = cp.zeros((n_paths,))

    for j in range(n_paths):
        v = v0
        I1 = 0.0  # ∫√v dZ
        I2 = 0.0  # ∫v dt

        for i in range(N_steps):
            dZ = cp.random.normal(0, cp.sqrt(dt))
            sqrt_v = cp.sqrt(cp.maximum(v, 1e-6))  # Ensure non-negative

            # Accumulate integrals
            I1 += sqrt_v * dZ
            I2 += v * dt

            # Update variance process (Euler-Maruyama with reflection)
            v_new = v + params['kappa'] * (params['theta'] - v) * dt + params['xi'] * sqrt_v * dZ
            v = cp.maximum(v_new, 0)  # Full truncation

        # Generate orthogonal Brownian component
        Z_ortho = cp.random.normal(0, 1.0)

        # Construct log-return with ALL components
        log_return = (r - 0.5 * I2) * T + \
                    params['rho'] * I1 + \
                    cp.sqrt(1 - params['rho']**2) * cp.sqrt(I2) * Z_ortho

        S_T = S0 * cp.exp(log_return)
        payoff_val = cp.maximum(S_T - K, 0) if kind == "call" else cp.maximum(K - S_T, 0)
        S_paths[j] = payoff_val

    S_paths_cpu = cp.asnumpy(S_paths)
    mean_payoff = np.exp(-r * T) * np.mean(S_paths_cpu)
    stderr = np.std(S_paths_cpu) / np.sqrt(n_paths)
    return mean_payoff, stderr

In [4]:
# Run and Print Results
print("\nRunning GPU Path Integral Monte Carlo...")
tau_corr = 100
# Calculate analytic price
analytic_price = black_scholes_analytic(S0, K, T, r, sigma, "call")
print(f"\nAnalytic BS Price: {analytic_price:.4f}")

# Run corrected PIMC
print("\nRunning Corrected GPU Path Integral Monte Carlo...")
bs_price, bs_error = metropolis_hastings_gpu(S0, K, T, r, sigma, N, n_sweeps, tau_corr, kind="call")
print(f"Corrected BS Path Integral MC: Price = {bs_price:.4f}, StdErr = {bs_error:.4f}")
print(f"Difference: {(bs_price - analytic_price):.4f} ({abs(bs_price - analytic_price)/analytic_price*100:.2f}%)")

print("\nRunning Heston MC simulation...")
params = {"kappa": 2.0, "theta": sigma**2, "xi": 0.4, "rho": -0.7}
v0 = sigma**2
heston_price, heston_err = simulate_heston_paths(S0, v0, params, T, N_steps, n_paths, K, kind="call")
print(f"Heston MC (Put): Price = {heston_price:.4f}, StdErr = {heston_err:.4f}")



Running GPU Path Integral Monte Carlo...

Analytic BS Price: 992.6682

Running Corrected GPU Path Integral Monte Carlo...
Corrected BS Path Integral MC: Price = 791.0119, StdErr = 0.0079
Difference: -201.6563 (20.31%)

Running Heston MC simulation...
Heston MC (Put): Price = 1053.3551, StdErr = 7.7662
